In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [3]:
tornados_outage_2019 = pd.read_csv('../merged/tornados_outage_2023.csv', parse_dates=['DATE'])


In [4]:
tornados_outage_2019.head()

,Unnamed: 0,DATE,AVGDV_max,LLDV_max,MXDV_max,MXDV_HEIGHT_max,DEPTH_max,MAX_SHEAR_max,MAX_SHEAR_HEIGHT_max,location,county,state,Event Month,power_outage
0,0,2023-01-01,36,49,56,7,6.5,29,6.6,"(32.39305, -110.68147)",Pima County,Arizona,NaN,False
1,1,2023-01-01,52,96,96,2,5.9,86,1.8,"(32.18141, -110.52664)",Cochise County,Arizona,NaN,False
2,2,2023-01-01,31,50,50,4,9.1,17,4.2,"(34.34357, -117.82177)",San Bernardino County,California,NaN,False
3,3,2023-01-01,35,62,62,6,8.3,21,6.4,"(34.29869, -117.62836)",San Bernardino County,California,NaN,False
4,4,2023-01-01,39,52,52,4,5.0,24,2.6,"(35.0269, -118.24596)",Kern County,California,NaN,False


In [7]:
tornados_grouped_by = tornados_outage_2019.groupby(['power_outage'])
tornados_balanced = tornados_grouped_by.apply(lambda x: x.sample(tornados_grouped_by.size().min()).reset_index(drop=True))
tornados_balanced =tornados_balanced.droplevel(['power_outage'])

/var/folders/n_/trtxk_px0d1d55c8b8n_7ksr0000gn/T/ipykernel_27864/1250737804.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tornados_balanced = tornados_grouped_by.apply(lambda x: x.sample(tornados_grouped_by.size().min()).reset_index(drop=True))


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
tornados_train, tornados_test = train_test_split(tornados_balanced.copy(),
                                              shuffle=True,
                                              random_state=123,
                                              test_size=.1,
                                              stratify=tornados_balanced.power_outage.values)

In [10]:
## import random forest classifier
from sklearn.ensemble import RandomForestClassifier

## import kfold
from sklearn.model_selection import StratifiedKFold

## import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score



In [11]:
## this will isolate the feature columns
features = tornados_train.columns[2: 9]

In [12]:
features

Index(['AVGDV_max', 'LLDV_max', 'MXDV_max', 'MXDV_HEIGHT_max', 'DEPTH_max',
       'MAX_SHEAR_max', 'MAX_SHEAR_HEIGHT_max'],
      dtype='object')

In [13]:
## set the number of CV folds
n_splits = 5

## Make the kfold object
kfold = StratifiedKFold(n_splits, 
                        random_state=216, 
                        shuffle=True)

In [14]:
max_depths = range(1, 11)
n_trees = [100, 500]

rf_accs = np.zeros((n_splits, len(max_depths), len(n_trees)))



for i,(train_index, test_index) in enumerate(kfold.split(tornados_train, tornados_train.power_outage)):
    tornados_tt = tornados_train.iloc[train_index]
    tornados_ho = tornados_train.iloc[test_index]

    for j, max_depth in enumerate(max_depths):
        for k, n_estimators in enumerate(n_trees):
            print(i,j,k)
            rf = RandomForestClassifier(n_estimators = n_estimators,
                                           max_depth = max_depth,
                                           max_samples = 0.8,
                                           random_state = 216)
                                           
            rf.fit(tornados_tt[features], tornados_tt.power_outage)
            
            pred = rf.predict(tornados_ho[features])
            
            rf_accs[i,j,k] = accuracy_score(tornados_ho.power_outage,  pred)

0 0 0
0 0 1
0 1 0
0 1 1
0 2 0
0 2 1
0 3 0
0 3 1
0 4 0
0 4 1
0 5 0
0 5 1
0 6 0
0 6 1
0 7 0
0 7 1
0 8 0
0 8 1
0 9 0
0 9 1
1 0 0
1 0 1
1 1 0
1 1 1
1 2 0
1 2 1
1 3 0
1 3 1
1 4 0
1 4 1
1 5 0
1 5 1
1 6 0
1 6 1
1 7 0
1 7 1
1 8 0
1 8 1
1 9 0
1 9 1
2 0 0
2 0 1
2 1 0
2 1 1
2 2 0
2 2 1
2 3 0
2 3 1
2 4 0
2 4 1
2 5 0
2 5 1
2 6 0
2 6 1
2 7 0
2 7 1
2 8 0
2 8 1
2 9 0
2 9 1
3 0 0
3 0 1
3 1 0
3 1 1
3 2 0
3 2 1
3 3 0
3 3 1
3 4 0
3 4 1
3 5 0
3 5 1
3 6 0
3 6 1
3 7 0
3 7 1
3 8 0
3 8 1
3 9 0
3 9 1
4 0 0
4 0 1
4 1 0
4 1 1
4 2 0
4 2 1
4 3 0
4 3 1
4 4 0
4 4 1
4 5 0
4 5 1
4 6 0
4 6 1
4 7 0
4 7 1
4 8 0
4 8 1
4 9 0
4 9 1


In [15]:
max_depths = range(1, 11)
n_trees = [100, 500]

rf_accs = np.zeros((n_splits, len(max_depths), len(n_trees)))
rf_prec = np.zeros((n_splits, len(max_depths), len(n_trees)))
rf_recall = np.zeros((n_splits, len(max_depths), len(n_trees)))



for i,(train_index, test_index) in enumerate(kfold.split(tornados_train, tornados_train.power_outage)):
    tornados_tt = tornados_train.iloc[train_index]
    tornados_ho = tornados_train.iloc[test_index]

    for j, max_depth in enumerate(max_depths):
        for k, n_estimators in enumerate(n_trees):
            print(i,j,k)
            rf = RandomForestClassifier(n_estimators = n_estimators,
                                           max_depth = max_depth,
                                           max_samples = 0.8,
                                           random_state = 216)
                                           
            rf.fit(tornados_tt[features], tornados_tt.power_outage)
            
            pred = rf.predict(tornados_ho[features])
            
            rf_accs[i,j,k] = accuracy_score(tornados_ho.power_outage,  pred)
            rf_prec[i,j,k]= precision_score(tornados_ho.power_outage,  pred)
            rf_recall[i,j,k]= recall_score(tornados_ho.power_outage,  pred)

0 0 0
0 0 1
0 1 0
0 1 1
0 2 0
0 2 1
0 3 0
0 3 1
0 4 0
0 4 1
0 5 0
0 5 1
0 6 0
0 6 1
0 7 0
0 7 1
0 8 0
0 8 1
0 9 0
0 9 1
1 0 0
1 0 1
1 1 0
1 1 1
1 2 0
1 2 1
1 3 0
1 3 1
1 4 0
1 4 1
1 5 0
1 5 1
1 6 0
1 6 1
1 7 0
1 7 1
1 8 0
1 8 1
1 9 0
1 9 1
2 0 0
2 0 1
2 1 0
2 1 1
2 2 0
2 2 1
2 3 0
2 3 1
2 4 0
2 4 1
2 5 0
2 5 1
2 6 0
2 6 1
2 7 0
2 7 1
2 8 0
2 8 1
2 9 0
2 9 1
3 0 0
3 0 1
3 1 0
3 1 1
3 2 0
3 2 1
3 3 0
3 3 1
3 4 0
3 4 1
3 5 0
3 5 1
3 6 0
3 6 1
3 7 0
3 7 1
3 8 0
3 8 1
3 9 0
3 9 1
4 0 0
4 0 1
4 1 0
4 1 1
4 2 0
4 2 1
4 3 0
4 3 1
4 4 0
4 4 1
4 5 0
4 5 1
4 6 0
4 6 1
4 7 0
4 7 1
4 8 0
4 8 1
4 9 0
4 9 1


We will do `GridSearchCV

In [16]:
max_index_prec = np.unravel_index(np.argmax(np.mean(rf_prec, axis=0)), 
                                       np.mean(rf_prec, axis=0).shape)


print(max_depths[max_index_prec[0]],n_trees[max_index_prec[1]])

1 500


In [17]:
max_index_recall = np.unravel_index(np.argmax(np.mean(rf_recall, axis=0)), 
                                       np.mean(rf_recall, axis=0).shape)


print(max_depths[max_index_recall[0]],n_trees[max_index_recall[1]])

10 500


In [18]:
np.mean(rf_recall, axis=0)

array([[0.48904392, 0.50055146],
       [0.49019999, 0.5005581 ],
       [0.49481098, 0.49481762],
       [0.49712976, 0.50518902],
       [0.51439107, 0.50633845],
       [0.53276194, 0.5155405 ],
       [0.53739951, 0.52245034],
       [0.53738622, 0.54315992],
       [0.56497907, 0.5558036 ],
       [0.56039466, 0.564999  ]])

In [19]:
np.mean(rf_prec, axis=0)

array([[0.6099288 , 0.61325044],
       [0.60268817, 0.60975058],
       [0.60494183, 0.60634576],
       [0.60775941, 0.60417887],
       [0.60482897, 0.59914381],
       [0.60448865, 0.59892362],
       [0.6002287 , 0.60845043],
       [0.59009158, 0.60369218],
       [0.59552216, 0.59626014],
       [0.59163509, 0.59404161]])

In [20]:
max_index = np.unravel_index(np.argmax(np.mean(rf_accs, axis=0)), 
                                       np.mean(rf_accs, axis=0).shape)


print(max_depths[max_index[0]],n_trees[max_index[1]])

7 500


In [21]:
## first import GridSearchCV
from sklearn.model_selection import GridSearchCV

In [22]:
grid_cv = GridSearchCV(RandomForestClassifier(), # first put the model object here
                          param_grid = {'max_depth':max_depths, # place the grid values for max_depth and
                                        'n_estimators':n_trees}, # and n_estimators here
                          scoring = 'accuracy', # put the metric we are trying to optimize here as a string, "accuracy"
                          cv = 5) # put the number of cv splits here

## you fit it just like a model
grid_cv.fit(tornados_train[features], tornados_train.power_outage)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 11), 'n_estimators': [100, 500]},
             scoring='accuracy')

In [23]:
## You can find the hyperparameter grid point that
## gave the best performance like so
## .best_params_
grid_cv.best_params_

{'max_depth': 2, 'n_estimators': 500}

In [24]:
## You can find the best score like so
## .best_score_
grid_cv.best_score_

0.5912534366822353

In [25]:
## Calling best_estimator_ returns the model with the 
## best avg cv performance after it has been refit on the
## entire data set
grid_cv.best_estimator_

RandomForestClassifier(max_depth=2, n_estimators=500)

In [26]:
grid_cv.best_estimator_.predict(tornados_train[features])

array([ True,  True,  True, ...,  True,  True, False])

In [27]:
## You can get all of the results with cv_results_
grid_cv.cv_results_

{'mean_fit_time': array([0.06079578, 0.29979858, 0.06946907, 0.33170719, 0.08191824,
        0.38570299, 0.08419914, 0.42407842, 0.09712667, 0.48360329,
        0.11133857, 0.55593686, 0.12418642, 0.59024963, 0.12999344,
        0.64057817, 0.14369068, 0.65724301, 0.1545836 , 0.71649814]),
 'std_fit_time': array([0.00154569, 0.00670258, 0.00047171, 0.01266551, 0.00192207,
        0.00967835, 0.00052486, 0.0100185 , 0.00119185, 0.01825704,
        0.00418531, 0.02396584, 0.00114123, 0.0155161 , 0.00430758,
        0.02827533, 0.00487682, 0.01823797, 0.00316144, 0.0437501 ]),
 'mean_score_time': array([0.00275197, 0.00959988, 0.00297756, 0.01007886, 0.00325561,
        0.01172805, 0.00317225, 0.01246762, 0.00375013, 0.01385646,
        0.00422897, 0.01679697, 0.00452724, 0.01688404, 0.00462751,
        0.01832576, 0.00516691, 0.0192482 , 0.00541096, 0.0206049 ]),
 'std_score_time': array([0.00016756, 0.00020197, 0.00014769, 0.00042378, 0.00016707,
        0.00028793, 0.00014699, 0.000492

Using either the `best_estimator_` fitted model or a refitted model according to our results from the `for` loop cross-validation we will find the feature importance scores. 

In [28]:
pd.DataFrame({'feature_importance_score':grid_cv.best_estimator_.feature_importances_},
                 index=features).sort_values('feature_importance_score',
                                                ascending=False)

,feature_importance_score
AVGDV_max,0.239727
MXDV_max,0.232867
DEPTH_max,0.147148
LLDV_max,0.145827
MAX_SHEAR_max,0.113392
MAX_SHEAR_HEIGHT_max,0.074315
MXDV_HEIGHT_max,0.046724


Bagging

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

In [30]:
pipe = Pipeline([('scale', StandardScaler()),('knn',KNeighborsClassifier())])
bag_pipe = BaggingClassifier(pipe, bootstrap = True, max_samples = 0.90)
bag_cv = GridSearchCV(bag_pipe, 
                          param_grid = {'estimator__knn__n_neighbors':[1,2,3], 
                                        'n_estimators':np.arange(1,100,10)}, 
                          scoring = 'accuracy', 
                          cv = 5)
bag_cv.fit(tornados_train[features], tornados_train.power_outage)

GridSearchCV(cv=5,
             estimator=BaggingClassifier(estimator=Pipeline(steps=[('scale',
                                                                    StandardScaler()),
                                                                   ('knn',
                                                                    KNeighborsClassifier())]),
                                         max_samples=0.9),
             param_grid={'estimator__knn__n_neighbors': [1, 2, 3],
                         'n_estimators': array([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91])},
             scoring='accuracy')

In [42]:
print(f"The best mean cv accuracy of {bag_cv.best_score_:.3f} was achieved using k = {bag_cv.best_estimator_.estimator['knn'].n_neighbors} and {bag_cv.best_estimator_.n_estimators} estimators")

The best mean cv accuracy of 0.571 was achieved using k = 3 and 51 estimators


In [44]:
single_pipe = Pipeline([('scale', StandardScaler()),('knn',KNeighborsClassifier(n_neighbors=3))])
single_cv = cross_validate(single_pipe, tornados_train[features], tornados_train.power_outage, cv = 5, scoring = 'accuracy')

In [45]:
print(f"The mean cv accuracy of a single kNN model with k=3 is {single_cv['test_score'].mean():.3f}")

The mean cv accuracy of a single kNN model with k=3 is 0.558


Model with `GridSearchCV`

In [46]:
model = grid_cv.best_estimator_

model.fit(tornados_train[features], tornados_train.power_outage)

RandomForestClassifier(max_depth=7)

In [47]:
accuracy_score(model.predict(tornados_train[features]), tornados_train.power_outage)

0.7869890616004606

In [48]:
accuracy_score(model.predict(tornados_test[features]), tornados_test.power_outage)

0.6683937823834197